In [1]:
# !wget https://huggingface.co/datasets/malaysia-ai/wikipedia/resolve/main/wikipedia-2023-10-01.jsonl

In [2]:
from transformers import AutoTokenizer
import json

tokenizer = AutoTokenizer.from_pretrained('mgoin/Meta-Llama-3-70B-Instruct-Marlin')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
def partition(text, size = 500, block_size = 4096):
    splitted = text.split()
    splitted = [' '.join(splitted[i: i + size]) for i in range(0, len(splitted), size)]
    temp = []
    results = []
    for t in splitted:
        tokenized = tokenizer(t)['input_ids']
        temp.extend(tokenized)
        while len(temp) >= block_size:
            block = temp[:block_size]
            temp = temp[block_size:]
            if len(block) == block_size:
                results.append(tokenizer.decode(block))
    if len(temp):
        results.append(tokenizer.decode(temp))
    return results

In [4]:
# !wget https://gist.githubusercontent.com/huseinzol05/98974ae8c6c7a65d4bc0af9f5003786a/raw/2e06e71ef7349a57bc58cc9913ae6bae1f9f8447/mp.py

In [5]:
import mp

In [3]:
from tqdm import tqdm

data = []
with open('wikipedia-2023-10-01.jsonl') as fopen:
    for l in tqdm(fopen):
        l = json.loads(l)
        if len(l['text']) < 50:
            continue
        data.append(l)
#         partitioned = partition(l['text'])
#         for p in partitioned:
#             data.append({
#                 'url': l['url'],
#                 'text': p
#             })

438332it [00:01, 316174.69it/s]


In [6]:
print(data[0]['text'])

Bahasa Melayu (Tulisan Jawi: بهاس ملايو; Rejang: ꤷꥁꤼ ꤸꥍꤾꤿꥈ) ialah salah satu daripada bahasa-bahasa Melayu-Polinesia di bawah keluarga bahasa Austronesia, yang merupakan bahasa rasmi di Brunei, Indonesia, Malaysia dan Singapura, serta dituturkan di Timor Leste dan sebahagian wilayah di Kemboja , Filipina dan Thailand. Jumlah penutur bahasa Melayu mencakupi lebih daripada 290 juta penutur (seramai 260 juta orang bertutur bahasa Indonesia) merentasi kawasan maritim Asia Tenggara. 
Sebagai salah satu daripada bahasa-bahasa yang paling luas digunakan di Asia Tenggara, bahasa Melayu mempunyai istilah perundangan yang berbeza di negara-negara terlibat bergantung pada sejarah dan budaya penggunaan bahasa Melayu di negara-negara tersebut. Di Malaysia, istilah "bahasa Melayu" ialah istilah "de jure" untuk pentakrifan rasmi bahasa kebangsaan negara Malaysia, manakala istilah "bahasa Malaysia" atau "bahasa Melayu Malaysia" seringkali digunakan mewakili perkara yang sama secara tidak formal di kal

In [7]:
def loop(texts):
    texts, _ = texts
    r = []
    for l in tqdm(texts):
        partitioned = partition(l['text'])
        for p in partitioned:
            r.append({
                'url': l['url'],
                'text': p
            })
            
    return r

In [8]:
r = loop((data[:10],0))

100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 73.31it/s]


In [9]:
import mp

r = mp.multiprocessing(data, loop, cores = 10)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [13]:
len(r)

364400

In [14]:
r[0]

{'url': 'https://ms.wikipedia.org/wiki?curid=15',
 'text': 'Bahasa Melayu (Tulisan Jawi: بهاس ملايو; Rejang: ꤷꥁꤼ ꤸꥍꤾꤿꥈ) ialah salah satu daripada bahasa-bahasa Melayu-Polinesia di bawah keluarga bahasa Austronesia, yang merupakan bahasa rasmi di Brunei, Indonesia, Malaysia dan Singapura, serta dituturkan di Timor Leste dan sebahagian wilayah di Kemboja, Filipina dan Thailand. Jumlah penutur bahasa Melayu mencakupi lebih daripada 290 juta penutur (seramai 260 juta orang bertutur bahasa Indonesia) merentasi kawasan maritim Asia Tenggara. Sebagai salah satu daripada bahasa-bahasa yang paling luas digunakan di Asia Tenggara, bahasa Melayu mempunyai istilah perundangan yang berbeza di negara-negara terlibat bergantung pada sejarah dan budaya penggunaan bahasa Melayu di negara-negara tersebut. Di Malaysia, istilah "bahasa Melayu" ialah istilah "de jure" untuk pentakrifan rasmi bahasa kebangsaan negara Malaysia, manakala istilah "bahasa Malaysia" atau "bahasa Melayu Malaysia" seringkali digun

In [15]:
with open('prepared-wikipedia-llama3.jsonl', 'w') as fopen:
    for d in r:
        fopen.write(f'{json.dumps(d)}\n')